%%% Imports

In [8]:

from __future__ import annotations
from time import time
from BrisbaneVPRDatasetSpeed import BrisbaneVPRDatasetSpeed
from VPRNetwork import VPRNetwork
from plotting import plot_confusion_matrix, plot_match_images, plot_gps
from constants import brisbane_event_traverses, brisbane_event_traverses_aliases
from utils import get_short_traverse_name

import os, sys
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy import signal
from datetime import datetime

import numpy as np

import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer.classifier import Rate


In [9]:
# Training settings
from cgi import test


epochs = 50
batch_size = 10

# Network settings
input_size = 34
threshold = 1.0

# Data settings
train_traverse = brisbane_event_traverses[0]
test_traverse = brisbane_event_traverses[2]
train_name = brisbane_event_traverses_aliases[get_short_traverse_name(train_traverse)]
test_name = brisbane_event_traverses_aliases[get_short_traverse_name(test_traverse)]
num_places = 45
start_dist = 200
place_gap = 100 #164/num_places # The streams run for approximately 164 seconds 
samples_per_sec = 1000
place_length = 27
max_spikes = None

# Sequencer settings
sequence_length = 3

# Plot settings
redo_plot = False
vmin = 0
vmax = 0.05

def transpose( matrix):
    if len(matrix) == 0:
        return []
    return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]


# Make a folder for the trained network
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# Use GPU
print(torch.cuda.is_available())
device = torch.device('cuda')

#---------------- Create the Network -----------------#

# Create the network
net = VPRNetwork(input_size, num_places, threshold=threshold).to(device)


True


In [10]:
# Load the data
training_set = BrisbaneVPRDatasetSpeed(train_traverse, train=True, place_length = place_length, place_gap=place_gap, num_places=num_places, start_dist=start_dist,samples_per_sec=samples_per_sec, max_spikes=max_spikes)
testing_set  = BrisbaneVPRDatasetSpeed(test_traverse, train=False, place_length = place_length, training_locations=training_set.place_locations, place_gap=place_gap, num_places=num_places, start_dist=start_dist, samples_per_sec=samples_per_sec, max_spikes=max_spikes)
            
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=True)

Loading training event streams ...
Adding GPS
Duration: 527.16s (which is 3432338 events)
Distance: 6308.49m (which is 3432338 events)
Place: 0
Distance: 27.00m (which is 16824 events)
Duration: 1.99s (which is 16824 events)


/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chopped_stream['distance'] -= start_distance


Place: 1
Distance: 27.00m (which is 12479 events)
Duration: 2.49s (which is 12479 events)
Place: 2
Distance: 26.99m (which is 6983 events)
Duration: 2.19s (which is 6983 events)
Place: 3
Distance: 26.99m (which is 12522 events)
Duration: 2.15s (which is 12522 events)
Place: 4
Distance: 26.99m (which is 16878 events)
Duration: 2.10s (which is 16878 events)
Place: 5
Distance: 27.00m (which is 10723 events)
Duration: 2.06s (which is 10723 events)
Place: 6
Distance: 27.00m (which is 9892 events)
Duration: 2.14s (which is 9892 events)
Place: 7
Distance: 27.00m (which is 23607 events)
Duration: 3.56s (which is 23607 events)
Place: 8
Distance: 27.00m (which is 20740 events)
Duration: 2.10s (which is 20740 events)
Place: 9
Distance: 27.00m (which is 20133 events)
Duration: 2.07s (which is 20133 events)
Place: 10
Distance: 27.00m (which is 18632 events)
Duration: 1.84s (which is 18632 events)
Place: 11
Distance: 27.00m (which is 17209 events)
Duration: 1.75s (which is 17209 events)
Place: 12
Di

/home/n10223622/snn-event-vpr/src/BrisbaneVPRDatasetSpeed.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chopped_stream['distance'] -= dist_at_start_time


Place: 1
Distance: 27.00m (which is 18131 events)
Duration: 2.63s (which is 18131 events)
Place: 2
Distance: 27.00m (which is 7087 events)
Duration: 2.26s (which is 7087 events)
Place: 3
Distance: 27.00m (which is 16439 events)
Duration: 2.07s (which is 16439 events)
Place: 4
Distance: 27.00m (which is 15092 events)
Duration: 1.94s (which is 15092 events)
Place: 5
Distance: 27.00m (which is 12726 events)
Duration: 1.81s (which is 12726 events)
Place: 6
Distance: 27.00m (which is 18308 events)
Duration: 1.95s (which is 18308 events)
Place: 7
Distance: 27.00m (which is 37342 events)
Duration: 3.52s (which is 37342 events)
Place: 8
Distance: 27.00m (which is 20890 events)
Duration: 1.96s (which is 20890 events)
Place: 9
Distance: 27.00m (which is 22289 events)
Duration: 1.80s (which is 22289 events)
Place: 10
Distance: 27.00m (which is 21889 events)
Duration: 1.67s (which is 21889 events)
Place: 11
Distance: 27.00m (which is 20625 events)
Duration: 1.71s (which is 20625 events)
Place: 12


In [11]:
# Define an optimiser 
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training the network
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)

# Create a training assistant object
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)


if redo_plot:
    for i, (input, label) in enumerate(train_loader): # training loop
        output = assistant.train(input, label)

else: 
    for epoch in range(epochs):

        for i, (input, label) in enumerate(train_loader): # training loop
            output = assistant.train(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        for i, (input, label) in enumerate(test_loader): # training loop
            output = assistant.test(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        if epoch%20 == 19: # cleanup display
            print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {stats}'))
            stats_str = str(stats).replace("| ", "\n")
            print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')

        if stats.testing.best_accuracy:
            torch.save(net.state_dict(), trained_folder + '/network.pt')
        stats.update()
        stats.save(trained_folder + '/')
        net.grad_flow(trained_folder + '/')

[Epoch  0/50] Train loss =     0.83873                        accuracy = 0.02222 | Test  loss =     0.60984                        accuracy = 0.06667
Accuracy was none
[Epoch  1/50] Train loss =     0.63105 (min =     0.83873)    accuracy = 0.02222 (max = 0.02222) | Test  loss =     1.06494 (min =     0.60984)
Accuracy was none

Accuracy was none
[Epoch  2/50] Train loss =     0.54270 (min =     0.63105)    accuracy = 0.13333 (max = 0.02222) | Test  loss =     1.29843 (min =     0.60984)    accuracy = 0.06667 (max = 0.06667)
Accuracy was none
[Epoch  3/50] Train loss =     0.52519 (min =     0.54270)    accuracy = 0.22222 (max = 0.13333) | Test  loss =     1.61526 (min =     0.60984)    accuracy = 0.04444 (max = 0.06667)
Accuracy was none
[Epoch  4/50] Train loss =     0.45968 (min =     0.52519)    accuracy = 0.31111 (max = 0.22222) | Test  loss =     1.74369 (min =     0.60984)    accuracy = 0.06667 (max = 0.06667)
Accuracy was none
[Epoch  5/50] Train loss =     0.41933 (min =     0

In [12]:
# import the best network during training 
net.load_state_dict(torch.load(trained_folder + '/network.pt'))
net.export_hdf5(trained_folder + '/network.net')

# Get the output for the input to each place
test_loader2  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=False)
rate = []
labels = []
for i, (input, label) in enumerate(test_loader2):
    output = net(input.to(device)) # Get network output
    #guesses = assistant.classifier(output).cpu().data.numpy() # get the predictions 
    rate.extend(Rate.rate(output).cpu().data.numpy()) # Get the firing rates for each place 
    labels.extend(label.cpu().data.numpy()) # Get place labels


# # Get rates in percentages of total
# for i in range(num_places):
#     sum = np.sum(rate[i])
#     if sum != 0:
#         rate[i] = np.divide(rate[i],sum)

# Make confusion matrix annotations
accuracy = 0
matches = []
annotations = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(rate[qryIndex])
    matches.append(max_idx)
    annotations[max_idx][qryIndex] = 'x'
    if max_idx ==qryIndex:
        accuracy += 1


#--------------- Apply a sequencer ------------------#
I = np.identity(sequence_length)
conv = signal.convolve2d(rate, I, mode='same')
print(np.shape(conv))

# Make confusion matrix annotations
accuracy_s = 0
matches_with_seq = []
annotations_s = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(conv[qryIndex])
    matches_with_seq.append(max_idx)
    annotations_s[max_idx][qryIndex] = 'x'
    if max_idx == qryIndex:
        accuracy_s += 1



(45, 45)


In [13]:

#--------------- Save Results ------------------#

# Make new folder for results
from tracemalloc import start
from plotting import plot_gps


time_stamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_path = "./../results/" + time_stamp
os.mkdir(results_path)


# Save query and match images 
images_path = results_path + "/matched_images"
images_path_seq = results_path + "/matched_images_seq"
os.mkdir(images_path)
os.mkdir(images_path_seq)

print('help')
plot_match_images(images_path + "/", matches, training_set.place_images, testing_set.place_images)
plot_match_images(images_path_seq + "/", matches_with_seq, training_set.place_images, testing_set.place_images)

# Save the confusion matrices
confusion_path = results_path + "/confusion_matrices"
os.mkdir(confusion_path)
rate = transpose(rate)
conv = transpose(conv)
output_path = confusion_path + "/confusion_matrix" 
output_path_s = confusion_path + "/confusion_matrix_seq" 
plot_confusion_matrix(rate, labels, annotations, output_path, vmin, vmax)
plot_confusion_matrix(conv, labels, annotations_s, output_path_s, vmin, vmax)

# Save GPS map'
gps_path = results_path + "/gps_locations"
plot_gps(gps_path, training_set.place_locations, testing_set.place_locations)

# Save test settings and accuracy
accuracy = accuracy/num_places
accuracy_s = accuracy_s/num_places


help


<Figure size 1296x288 with 0 Axes>

<Figure size 1296x288 with 0 Axes>

In [14]:

log_path = results_path + "/log.txt"
log_string  = """
---- DATA SETTINGS ---- 
Training datasets = {}
Testing datasets = {} 
# Places = {}
Start dist = {} [m]
Place gap = {} [m]
Place length = {} [m]
Samples per second = {} 
Max spikes per sample = {}
---- NETWORK SETTINGS ---- 
Input size = {}x{} 
Threshold = {}
---- TRAINING SETTINGS ---- 
Epochs = {}
Batch size = {}
---- SEQUENCER SETTINGS ----
Sequence lengtth = {} 
---- RESULTS ----
Accuracy = {}
Accuracy (sequencer) = {}
""".format(train_name,test_name,num_places,start_dist,place_gap,place_length,
                samples_per_sec,max_spikes,input_size,input_size,threshold,epochs,
                batch_size,sequence_length,accuracy,accuracy_s)
f = open(log_path,'w')
f.write(log_string)
f.close()

print("The accuracy of the network is: " + str(accuracy))
print("The accuracy with a sequencer is: " + str(accuracy_s))

The accuracy of the network is: 0.17777777777777778
The accuracy with a sequencer is: 0.35555555555555557
